In [1]:
from datastore import DataStore as ds


#ds.config.enable_debug()


nat = ds.from_file('../tests/dataset/users.csv')

nat = nat.filter(nat.age < 35)

nat['age_minus_10'] = nat['age'] - 10

nat = nat.add_prefix('col_').filter(nat.col_age > 25).filter(nat.col_country == 'USA')

nat['doubled'] = nat['col_age'] * 2

nat2 = nat.sql("doubled > 35")
nat2["float_age"] = nat2.doubled.cast("Float64")
print('Executing...')
# nat2.explain()

nat2["col_age"]
#df = nat2.to_df()
#print(df)

Executing...


0    28
1    31
2    29
Name: col_age, dtype: int64

In [2]:
nat2["col_name"].str.upper()
nat2.to_df()["col_name"].str.upper()
nat2["col_name"].upper()


AttributeError: 'ColumnExpr' object has no attribute 'upper'